<a href="https://colab.research.google.com/github/yangswei112/Text-to-Morse-Code/blob/master/buzzlive_live_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pendulum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 24.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pendulum
from helperfunction import day_to_num, month_to_num, rupiah_format, views_revenue_graph, views_likes_graph

## Class

In [ ]:
class Reporting:

    def __init__(self, df, total_duration):
        self.df = df
        self.total_duration = total_duration

    def preprocessing(self):
        new_header = self.df.iloc[0] #grab the first row for the header
        self.df = self.df[1:] #take the data less the header row
        self.df = self.df[:-1] #drop the last row
        self.df.columns = new_header #set the header row as the df header
        self.df.drop(['No', 'Jobs', 'Merchant', 'Host', 'Link', 'Upload', 'Duration',
          'Platform', 'Status', 'Action'], axis=1, inplace=True)
        self.df.fillna(0, inplace=True)
        self.df.Date = pd.to_datetime(self.df.Date)
        self.df['Hour'] = self.df.Date.apply(lambda x: x.strftime("%I:%M %p"))
        self.df['Day'] = self.df.Date.apply(lambda x: x.strftime('%A'))
        self.df['Week'] = self.df['Date'].apply(lambda x: pendulum.parse(str(x.date())).week_of_month)
        self.df['Month'] = self.df['Date'].apply(lambda x: x.strftime("%B"))

    def hourly_graph(self):
        hourly = self.df[['Views', 'Likes', 'Total Sales']].groupby(self.df['Hour']).sum().reset_index()
        hourly['HourSorter'] = pd.to_datetime(hourly.Hour)
        hourly.sort_values(by='HourSorter', inplace=True)
        views_revenue_graph(hourly, 1500, 700, t="Hourly Total Sales and Viewers in December 2023")
        views_likes_graph(hourly, t="Hourly Total Views vs. Likes in December 2023")

    def daily_graph(self):
        daily = self.df[['Views', 'Likes', 'Total Sales']].groupby(self.df['Day']).sum().reset_index()
        daily['DaySorter'] = daily.Day.apply(lambda x: day_to_num(x))
        daily.sort_values(by='DaySorter', inplace=True)
        views_revenue_graph(daily, 1500, 700, t="Daily Total Sales and Viewers in December 2023")
        views_likes_graph(daily, t="Daily Total Views vs. Likes in December 2023")

    def weekly_graph(self):
        weekly = self.df[['Views', 'Likes', 'Total Sales']].groupby(self.df['Week']).sum().reset_index()
        views_revenue_graph(weekly, 1500, 700, t="Weekly Total Sales and Viewers in December 2023")
        views_likes_graph(weekly, t="Weekly Total Views vs. Likes in December 2023")
        return weekly

    def monthly_graph(self):
        monthly = self.df[['Views', 'Likes', 'Total Sales']].groupby(self.df['Month']).sum().reset_index()
        monthly['MonthSorter'] = monthly['Month'].apply(lambda x: month_to_num(x))
        monthly = monthly.sort_values(by='MonthSorter')
        views_revenue_graph(monthly, 1500, 700, t="Monthly Total Sales and Viewers in December 2023")
        views_likes_graph(monthly, t="Monthly Total Views vs. Likes in December 2023")

    def overal_performance(self, ptd, ptv, ptl, ptr):
        total_viewers = sum(self.df.Views)
        avg_viewers = total_viewers/self.total_duration
        total_likes = sum(self.df.Likes)
        avg_likes = total_likes/self.total_duration
        total_revenue = sum(self.df['Total Sales'])
        avg_revenue = total_revenue/self.total_duration

        self.prev_total_duration = ptd
        self.prev_total_viewers = ptv
        prev_avg_viewers = self.prev_total_viewers/self.prev_total_duration
        self.prev_total_likes = ptl
        prev_avg_likes = self.prev_total_likes/self.prev_total_duration
        self.prev_total_revenue = ptr
        prev_avg_revenue = self.prev_total_revenue/self.prev_total_duration

        growth = pd.DataFrame({'Growth Total Duration': [round(((self.total_duration-self.prev_total_duration)/self.prev_total_duration)*100,2)],
                              'Growth Viewers': [round(((total_viewers-self.prev_total_viewers)/self.prev_total_viewers)*100,2)],
                              'Growth Avg. Viewers': [round(((avg_viewers-prev_avg_viewers)/prev_avg_viewers)*100,2)],
                              'Growth Likes': [round(((total_likes-self.prev_total_likes)/self.prev_total_likes)*100,2)],
                              'Growth Avg. Likes': [round(((avg_likes-prev_avg_likes)/prev_avg_likes)*100,2)],
                              'Growth Revenue': [round(((total_revenue-self.prev_total_revenue)/self.prev_total_revenue)*100,2)],
                              'Growth Avg. Revenue': [round(((avg_revenue-prev_avg_revenue)/prev_avg_revenue)*100,2)]})

        overall = pd.DataFrame({'Total Duration': [f"{self.total_duration} ({np.squeeze(growth['Growth Total Duration'].values)}%)"],
                                'Viewers': [f"{total_viewers} ({np.squeeze(growth['Growth Viewers'].values)}%)"],
                                'Avg. Viewers': [f"{avg_viewers} ({np.squeeze(growth['Growth Avg. Viewers'].values)}%)"],
                                'Likes': [f"{total_likes} ({np.squeeze(growth['Growth Likes'].values)}%)"],
                                'Avg. Likes': [f"{avg_likes} ({np.squeeze(growth['Growth Avg. Likes'].values)}%)"],
                                'Revenue': [f"{rupiah_format(total_revenue)} ({np.squeeze(growth['Growth Revenue'].values)}%)"],
                                'Avg. Revenue': [f"{rupiah_format(avg_revenue)} ({np.squeeze(growth['Growth Avg. Revenue'].values)}%)"]})

        return overall

In [ ]:
df = pd.read_excel("/content/Mohorahora December 2023.xlsx")
report_1 = Reporting(df, 40)
report_1.preprocessing()
report_1.hourly_graph()
report_1.daily_graph()
report_1.weekly_graph()
report_1.monthly_graph()
# report_1.overal_performance(ptd=60, ptv=8718, ptl=88849, ptr=9209749)

In [ ]:
df = pd.read_excel("/content/Mohorahora December 2023.xlsx")
report_1 = Reporting(df, 40)
report_1.preprocessing()
print(report_1.weekly_graph())

0  Week  Views  Likes  Total Sales
0     2     52   2600            0
1     3   1811   7352       677674
2     4    159   5446       278912
3     5    434   2512       432657


## Reporting

### Load Data & Data Preprocessing

In [ ]:
df = pd.read_excel("/content/MOC November 2023.xlsx")
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df = df[:-1] #drop the last row
df.columns = new_header #set the header row as the df header
df.head()

,No,Jobs,Merchant,Host,Date,Link,Upload,Duration,Platform,Views,Likes,Total Sales,Status,Action
1,1,MOC X Jessica Nov 2023,BuzzLive,Jessica Octavia Pakpahan,30 Nov 2023 20:00,Open Link : https://shopee.co.id/universal-lin...,https://buzzlive.co/assets/images/jobs/upload/...,03 hours,shopee,1200,6100,4600979,Done,NaN
2,2,MOC X Ferla Nov 2023,BuzzLive,Ferla septiana,30 Nov 2023 11:30,Open Link : https://shopee.co.id/universal-lin...,https://buzzlive.co/assets/images/jobs/upload/...,03 hours,shopee,788,1000,4330940,Done,NaN
3,3,MOC X Jessica Nov 2023,BuzzLive,Jessica Octavia Pakpahan,29 Nov 2023 20:00,Open Link : https://shopee.co.id/universal-lin...,https://buzzlive.co/assets/images/jobs/upload/...,03 hours,shopee,1200,4300,5738138,Done,NaN
4,4,MOC X Ferla Nov 2023,BuzzLive,Ferla septiana,29 Nov 2023 11:30,Open Link : https://shopee.co.id/universal-lin...,https://buzzlive.co/assets/images/jobs/upload/...,03 hours,shopee,667,2100,2301825,Done,NaN
5,5,MOC X Ferla Nov 2023,BuzzLive,Ferla septiana,28 Nov 2023 11:30,Open Link : https://shopee.co.id/universal-lin...,https://buzzlive.co/assets/images/jobs/upload/...,03 hours,shopee,781,662,2140450,Done,NaN


In [ ]:
df.columns

Index(['No', 'Jobs', 'Merchant', 'Host', 'Date', 'Link', 'Upload', 'Duration',
       'Platform', 'Views', 'Likes', 'Total Sales', 'Status', 'Action'],
      dtype='object', name=0)

In [ ]:
df.drop(['No', 'Jobs', 'Merchant', 'Host', 'Link', 'Upload', 'Duration',
       'Platform', 'Status', 'Action'], axis=1, inplace=True)
df.head()

,Date,Views,Likes,Total Sales
1,30 Nov 2023 20:00,1200,6100,4600979
2,30 Nov 2023 11:30,788,1000,4330940
3,29 Nov 2023 20:00,1200,4300,5738138
4,29 Nov 2023 11:30,667,2100,2301825
5,28 Nov 2023 11:30,781,662,2140450


In [ ]:
df.isna().sum()

0
Date           0
Views          0
Likes          0
Total Sales    0
dtype: int64

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
df.head()

,Date,Views,Likes,Total Sales
1,2023-11-30 20:00:00,1200,6100,4600979
2,2023-11-30 11:30:00,788,1000,4330940
3,2023-11-29 20:00:00,1200,4300,5738138
4,2023-11-29 11:30:00,667,2100,2301825
5,2023-11-28 11:30:00,781,662,2140450


In [ ]:
df['Day'] = df.Date.apply(lambda x: x.strftime('%A'))
df.head()

,Date,Views,Likes,Total Sales,Day
1,2023-11-30 20:00:00,1200,6100,4600979,Thursday
2,2023-11-30 11:30:00,788,1000,4330940,Thursday
3,2023-11-29 20:00:00,1200,4300,5738138,Wednesday
4,2023-11-29 11:30:00,667,2100,2301825,Wednesday
5,2023-11-28 11:30:00,781,662,2140450,Tuesday


In [ ]:
df['Hour'] = df.Date.apply(lambda x: x.strftime("%I:%M %p"))
df.head()

,Date,Views,Likes,Total Sales,Day,Hour
1,2023-11-30 20:00:00,1200,6100,4600979,Thursday,08:00 PM
2,2023-11-30 11:30:00,788,1000,4330940,Thursday,11:30 AM
3,2023-11-29 20:00:00,1200,4300,5738138,Wednesday,08:00 PM
4,2023-11-29 11:30:00,667,2100,2301825,Wednesday,11:30 AM
5,2023-11-28 11:30:00,781,662,2140450,Tuesday,11:30 AM


In [ ]:
df['Week'] = df['Date'].apply(lambda x: pendulum.parse(str(x.date())).week_of_month)
df.head()

,Date,Views,Likes,Total Sales,Day,Hour,Week
1,2023-11-30 20:00:00,1200,6100,4600979,Thursday,08:00 PM,5
2,2023-11-30 11:30:00,788,1000,4330940,Thursday,11:30 AM,5
3,2023-11-29 20:00:00,1200,4300,5738138,Wednesday,08:00 PM,5
4,2023-11-29 11:30:00,667,2100,2301825,Wednesday,11:30 AM,5
5,2023-11-28 11:30:00,781,662,2140450,Tuesday,11:30 AM,5


In [ ]:
df['Month'] = df['Date'].apply(lambda x: x.strftime("%B"))
df.head()

,Date,Views,Likes,Total Sales,Day,Hour,Week,Month
1,2023-11-30 20:00:00,1200,6100,4600979,Thursday,08:00 PM,5,November
2,2023-11-30 11:30:00,788,1000,4330940,Thursday,11:30 AM,5,November
3,2023-11-29 20:00:00,1200,4300,5738138,Wednesday,08:00 PM,5,November
4,2023-11-29 11:30:00,667,2100,2301825,Wednesday,11:30 AM,5,November
5,2023-11-28 11:30:00,781,662,2140450,Tuesday,11:30 AM,5,November


### Hourly Graph

In [ ]:
hourly = df[['Views', 'Likes', 'Total Sales']].groupby(df['Hour']).sum().reset_index()
hourly['HourSorter'] = pd.to_datetime(hourly.Hour)
hourly.sort_values(by='HourSorter', inplace=True)
hourly.head()

,Hour,Views,Likes,Total Sales,HourSorter
4,08:00 AM,990,955,3399622,2023-12-21 08:00:00
6,11:30 AM,6725,8853,40388994,2023-12-21 11:30:00
8,12:00 PM,644,1100,4757757,2023-12-21 12:00:00
0,01:00 PM,2617,1984,10618420,2023-12-21 13:00:00
1,03:00 PM,180,27,867330,2023-12-21 15:00:00


In [ ]:
hourly_bar = px.bar(hourly,
        x = 'Hour',
        y = 'Total Sales',
        text = hourly['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Views',
        color_continuous_scale='Greens',
        title='Hourly Total Sales and Viewers in November 2023')

hourly_bar.update_layout(xaxis_title='Hour',
                    width = 1500,
                    height = 700,
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
hourly_bar.show()

In [ ]:
hourly_bar = px.bar(hourly,
        x = 'Hour',
        y = 'Total Sales',
        text = hourly['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Likes',
        color_continuous_scale='Blues',
        title='Hourly Total Sales and Likes in November 2023')

hourly_bar.update_layout(xaxis_title='Hour',
                    width = 1500,
                    height = 700,
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
hourly_bar.show()

In [ ]:
fig = go.Figure(
    data=go.Bar(
        x = hourly['Hour'],
        y = hourly['Views'],
        text = hourly['Views'],
        name = "Total Views",
    )
)

fig.add_trace(
    go.Scatter(
        x=hourly['Hour'],
        y=hourly['Likes'],
        yaxis="y2",
        name="Total Likes",
        marker=dict(color="crimson"),
    )
)

fig.update_layout(
    title = 'Hourly Views vs. Likes in November 2023',
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Total Views"),
        side="left",
    ),
    yaxis2=dict(
        title=dict(text="Total Likes"),
        side="right",
        overlaying="y",
        tickmode="sync",
    ),
)

fig.show()

### Daily Graph

In [ ]:
daily = df[['Views', 'Likes', 'Total Sales']].groupby(df['Day']).sum().reset_index()
daily['DaySorter'] = daily.Day.apply(lambda x: day_to_num(x))
daily.sort_values(by='DaySorter', inplace=True)
daily.head()

,Day,Views,Likes,Total Sales,DaySorter
1,Monday,3674,10192,15573885,0
5,Tuesday,2315,2837,8983232,1
6,Wednesday,6162,11579,32188029,2
4,Thursday,5379,13518,24713086,3
0,Friday,3619,10900,21829758,4


In [ ]:
daily_bar = px.bar(daily,
        x = 'Day',
        y = 'Total Sales',
        text = daily['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Views',
        color_continuous_scale='Greens',
        title='Daily Total Sales and Viewers in November 2023')

daily_bar.update_layout(xaxis_title='Day',
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
daily_bar.show()

In [ ]:
daily_bar = px.bar(daily,
        x = 'Day',
        y = 'Total Sales',
        text = daily['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Likes',
        color_continuous_scale='Blues',
        title='Daily Total Sales and Likes in November 2023')

daily_bar.update_layout(xaxis_title='Day',
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
daily_bar.show()

In [ ]:
fig = go.Figure(
    data=go.Bar(
        x = daily['Day'],
        y = daily['Views'],
        text = daily['Views'],
        name = "Total Views",
    )
)

fig.add_trace(
    go.Scatter(
        x=daily['Day'],
        y=daily['Likes'],
        yaxis="y2",
        name="Total Likes",
        marker=dict(color="crimson"),
    )
)

fig.update_layout(
    title = 'Daily Views vs. Likes in November 2023',
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Total Views"),
        side="left",
    ),
    yaxis2=dict(
        title=dict(text="Total Likes"),
        side="right",
        overlaying="y",
        tickmode="sync",
    ),
)

fig.show()

### Weekly Graph

In [ ]:
weekly = df[['Views', 'Likes', 'Total Sales']].groupby(df['Week']).sum().reset_index()
weekly

,Week,Views,Likes,Total Sales
0,1,5285,3843,37823249
1,2,6263,5149,28134333
2,3,5103,6281,27968391
3,4,8397,26266,41039669
4,5,6306,20696,28405762


In [ ]:
weekly_bar = px.bar(weekly,
        x = 'Week',
        y = 'Total Sales',
        text = weekly['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Views',
        color_continuous_scale='Greens',
        title='Weekly Total Sales and Viewers in November 2023')

weekly_bar.update_layout(xaxis_title='Week',
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
weekly_bar.show()

In [ ]:
weekly_bar = px.bar(weekly,
        x = 'Week',
        y = 'Total Sales',
        text = weekly['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Likes',
        color_continuous_scale='Blues',
        title='Weekly Total Sales and Likes in November 2023')

weekly_bar.update_layout(xaxis_title='Week',
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
weekly_bar.show()

In [ ]:
fig = go.Figure(
    data=go.Bar(
        x = weekly['Week'],
        y = weekly['Views'],
        text = weekly['Views'],
        name = "Total Views",
    )
)

fig.add_trace(
    go.Scatter(
        x=weekly['Week'],
        y=weekly['Likes'],
        yaxis="y2",
        name="Total Likes",
        marker=dict(color="crimson"),
    )
)

fig.update_layout(
    width=700,
    height=500,
    title = 'Weekly Views vs. Likes in 2023',
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Total Views"),
        side="left",
    ),
    yaxis2=dict(
        title=dict(text="Total Likes"),
        side="right",
        overlaying="y",
        tickmode="sync",
    ),
)

fig.show()

### Monthly Graph

In [ ]:
monthly = df[['Views', 'Likes', 'Total Sales']].groupby(df['Month']).sum().reset_index()
monthly['MonthSorter'] = monthly['Month'].apply(lambda x: month_to_num(x))
monthly = monthly.sort_values(by='MonthSorter')
monthly

,Month,Views,Likes,Total Sales,MonthSorter
0,November,31354,62235,163371404,10


In [ ]:
monthly_bar = px.bar(monthly,
        x = 'Month',
        y = 'Total Sales',
        text = monthly['Total Sales'].apply(lambda x: rupiah_format(x)),
        color = 'Views',
        color_continuous_scale='Greens',
        title='Monthly Total Sales and Viewers in 2023')

monthly_bar.update_layout(xaxis_title='Month',
                    width = 900,
                    height = 600,
                    # coloraxis_showscale=False,
                    yaxis_title='Total Sales')
monthly_bar.show()

In [ ]:
fig = go.Figure(
    data=go.Bar(
        x = monthly['Month'],
        y = monthly['Views'],
        text = monthly['Views'],
        name = "Total Views",
    )
)

fig.add_trace(
    go.Scatter(
        x=monthly['Month'],
        y=monthly['Likes'],
        yaxis="y2",
        name="Total Likes",
        marker=dict(color="crimson"),
    )
)

fig.update_layout(
    width=700,
    height=500,
    title = 'Monthly Views vs. Likes in 2023',
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Total Views"),
        side="left",
    ),
    yaxis2=dict(
        title=dict(text="Total Likes"),
        side="right",
        overlaying="y",
        tickmode="sync",
    ),
)

fig.show()

### Overall Performance & Comparison

In [ ]:
total_duration = 144
total_viewers = sum(df.Views)
avg_viewers = total_viewers/total_duration
total_likes = sum(df.Likes)
avg_likes = total_likes/total_duration
total_revenue = sum(df['Total Sales'])
avg_revenue = total_revenue/total_duration

In [ ]:
prev_total_duration = 117
prev_total_viewers = 26229
prev_avg_viewers = prev_total_viewers/prev_total_duration
prev_total_likes = 24538
prev_avg_likes = prev_total_likes/prev_total_duration
prev_total_revenue = 64190405
prev_avg_revenue = prev_total_revenue/prev_total_duration

growth = pd.DataFrame({'Growth Total Duration': [round(((total_duration-prev_total_duration)/prev_total_duration)*100,2)],
                      'Growth Viewers': [round(((total_viewers-prev_total_viewers)/prev_total_viewers)*100,2)],
                      'Growth Avg. Viewers': [round(((avg_viewers-prev_avg_viewers)/prev_avg_viewers)*100,2)],
                      'Growth Likes': [round(((total_likes-prev_total_likes)/prev_total_likes)*100,2)],
                      'Growth Avg. Likes': [round(((avg_likes-prev_avg_likes)/prev_avg_likes)*100,2)],
                      'Growth Revenue': [round(((total_revenue-prev_total_revenue)/prev_total_revenue)*100,2)],
                      'Growth Avg. Revenue': [round(((avg_revenue-prev_avg_revenue)/prev_avg_revenue)*100,2)]})

growth

,Growth Total Duration,Growth Viewers,Growth Avg. Viewers,Growth Likes,Growth Avg. Likes,Growth Revenue,Growth Avg. Revenue
0,23.08,19.54,-2.87,153.63,106.07,154.51,106.79


In [ ]:
overall = pd.DataFrame({'Total Duration': [f"{total_duration} ({np.squeeze(growth['Growth Total Duration'].values)}%)"],
                        'Viewers': [f"{total_viewers} ({np.squeeze(growth['Growth Viewers'].values)}%)"],
                        'Avg. Viewers': [f"{avg_viewers} ({np.squeeze(growth['Growth Avg. Viewers'].values)}%)"],
                        'Likes': [f"{total_likes} ({np.squeeze(growth['Growth Likes'].values)}%)"],
                        'Avg. Likes': [f"{avg_likes} ({np.squeeze(growth['Growth Avg. Likes'].values)}%)"],
                        'Revenue': [f"{rupiah_format(total_revenue)} ({np.squeeze(growth['Growth Revenue'].values)}%)"],
                        'Avg. Revenue': [f"{rupiah_format(avg_revenue)} ({np.squeeze(growth['Growth Avg. Revenue'].values)}%)"]})

overall

,Total Duration,Viewers,Avg. Viewers,Likes,Avg. Likes,Revenue,Avg. Revenue
0,144 (23.08%),31354 (19.54%),217.73611111111111 (-2.87%),62235 (153.63%),432.1875 (106.07%),"Rp163,371,404 (154.51%)","Rp1,134,524 (106.79%)"


## Testing

In [ ]:
hourly = df[['Views', 'Likes', 'Total Sales']].groupby(df['Hour']).sum().reset_index()
hourly['HourSorter'] = pd.to_datetime(hourly.Hour)
hourly.sort_values(by='HourSorter', inplace=True)
hourly.head()

,Hour,Views,Likes,Total Sales,HourSorter
4,08:00 AM,990,955,3399622,2023-12-21 08:00:00
6,11:30 AM,6725,8853,40388994,2023-12-21 11:30:00
8,12:00 PM,644,1100,4757757,2023-12-21 12:00:00
0,01:00 PM,2617,1984,10618420,2023-12-21 13:00:00
1,03:00 PM,180,27,867330,2023-12-21 15:00:00


In [ ]:
hourly.columns[0]

'Hour'

In [ ]:
def views_revenue_graph(data):

    bar = px.bar(data,
                  x = data.columns[0],
                  y = 'Total Sales',
                  text = data['Total Sales'].apply(lambda x: rupiah_format(x)),
                  color = 'Views',
                  color_continuous_scale='Greens',
                  title=f'{data.columns[0]}ly Total Sales and Viewers in November 2023')
    bar.update_layout(xaxis_title=data.columns[0],
                          width = 1500,
                          height = 700,
                          # coloraxis_showscale=False,
                          yaxis_title='Total Sales')
    bar.show()

def views_likes_graph(data):
    fig = go.Figure(
        data=go.Bar(
            x = data.iloc[:,0],
            y = data['Views'],
            text = data['Views'],
            name = "Total Views",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=data.iloc[:,0],
            y=hourly['Likes'],
            yaxis="y2",
            name="Total Likes",
            marker=dict(color="crimson"),
        )
    )

    fig.update_layout(
        title = f'{data.columns[0]}ly Views vs. Likes in November 2023',
        legend=dict(orientation="h"),
        yaxis=dict(
            title=dict(text="Total Views"),
            side="left",
        ),
        yaxis2=dict(
            title=dict(text="Total Likes"),
            side="right",
            overlaying="y",
            tickmode="sync",
        ),
    )

    fig.show()

In [ ]:
views_revenue_graph(hourly)

In [ ]:
views_likes_graph(hourly)